In [54]:
#  make jupyter notebook dynamically reload modules
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
from scrap.scrap_apa import download_pdf
import pandas as pd
import requests
import os

def check_and_process_url(row: pd.Series, ind: int) -> None:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4343.0 Safari/537.36",
    }
    request_timeout = 10
    # try-except block to handle exceptions
    try:
        response = requests.get(
            row['link_pdf'], headers=headers, timeout=request_timeout, stream=True  # type: ignore
        )
        if response.status_code == 200:
            print("View info url is valid!")
            pdf_filename = os.path.join(
                "scrap",
                "pdfs",
                # fstring to format the index with leading zeros 3 digits
                f"{row['Index']:03d}-{row["University"]}-{row["City"]}.pdf",
                
            )
            has_pdf =  download_pdf(response, pdf_filename)
            if has_pdf:
                print(f"Downloaded pdf file: {pdf_filename}")
            else:
                print(f"Failed to download pdf file: {pdf_filename}")
        else:
            has_pdf = False
            print(f"The url is not valid with code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to check url: {e}")
        has_pdf = False
    return has_pdf

## download pdf for link

In [ ]:
import numpy as np
excel_file = "scrap/result/apa_programs_licensure.xlsx"
df = pd.read_excel(excel_file)
for file_name, row in df.iterrows():
    if row['link_pdf'] is not np.nan:
        # print(f"Processing row {ind}")
        print(row['link_pdf'])
        has_pdf = check_and_process_url(row, file_name)
        df.loc[file_name, "has_pdf"] = has_pdf
        if has_pdf:
            df.loc[file_name, "pdf_filename"] = f"{row['Index']:03d}-{row['University']}-{row['City']}.pdf"
# save the updated dataframe
df.to_excel(excel_file, index=False)


# download html table

In [57]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import os

table_keywords = {
    "Mean number of years to complete the program" : "time_to_complete",
    "Tuition for full-time students" : "tuition",
    "Students who obtained APA/CPA" : "internship1",
    "Students who sought or applied for internships" : "internship2",
    "Students for whom this" : "attrition",
    "Licensure percentage" : "licensure",
}

def get_table(url:str, file_name) -> None:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4343.0 Safari/537.36",
    }
    request_timeout = 10
    # try-except block to handle exceptions
    try:
        response = requests.get(
            url, headers=headers, timeout=request_timeout, stream=True  
        )
        if response.status_code == 200:
            print("View info url is valid!")
            soup = BeautifulSoup(response.content, 'html.parser')
            tables = find_all_tables(soup, file_name)
            return True
        else:
            print(f"The url is not valid with code: {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"Failed to check url: {e}")
    return False

def find_all_tables(soup: BeautifulSoup, file_name) -> list:
    tables = soup.find_all("table")
    print(f"Found {len(tables)} tables in the html file")
    if len(tables) == 0:
        print(f"No tables found in the html file")
        return False
    else:
        output_txt = os.path.join('scrap',"html_table", f"{file_name}.txt")
        with open(output_txt, "a") as out:
            for table in tables:
                # convert html table to pandas dataframe
                try:
                    print("="*100)
                    print("Converting html table to pandas dataframe")
                    print(str(table))
                    df = pd.read_html(str(table).replace('%', ''))[0]
                    assert isinstance(df, pd.DataFrame)
                    # remove the '\n' in the dataframe
                    df = df.replace(r'\n','', regex=True)
                    # search for the keywords in the table
                    out.write("="*100)
                    out.write("\n")
                    for key, title in table_keywords.items():
                        if key in df.to_string():
                            out.write('title: ')
                            out.write(title)
                            out.write("\n")
                    out.write("="*100)
                    out.write("\n")
                    # new line for each table            
                    out.write(df.to_string())   
                    out.write("\n")
                    out.write("="*100)
                    out.write("\n")
                    return True
                except Exception as e:
                    print(f"Failed to convert html table to pandas dataframe: {e}")
                    return False


In [51]:
html_file = "/Users/wenqiangfang/PythonProject/web_scraping/scrap/pdfs/005.html"
with open(html_file, "r") as f:
    html = f.read()
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all("table")

In [52]:
for tab in tables:
    print("="*100)
    print(tab)
    try:
        df = pd.read_html(str(tab).replace('%', ''))[0]
    except Exception as e:
        print(f"Failed to convert html table to pandas dataframe: {e}")
    print("="*50)
    print(df.to_string())

<table><tbody><tr><th scope="col" width="70%">Description</th>
<th scope="col" width="30%"> </th>
</tr><tr><td>
<p>Does the program or institution require students, trainees, and/or staff (faculty) to comply with specific policies or practices related to the institution’s affiliation or purpose? Such policies or practices may include, but are not limited to, admissions, hiring, retention policies, and/or requirements for completion that express mission and values?</p>
</td>
<td>
<p>_______  Yes</p>
<p>___X___  No</p>
</td>
</tr></tbody></table>
                                                                                                                                                                                                                                                                                                                                                         Description              Unnamed: 1
0  Does the program or institution require students, trainees, and/o

In [58]:
excel_file = "scrap/result/apa_programs_licensure.xlsx"
df = pd.read_excel(excel_file)
row = df.loc[5]

file_name = f"{row['Index']:03d}-{row['University']}-{row['City']}"
print(file_name)
tables = get_table(row["info_href"], file_name)
# print(tables)

005-University of San Francisco-San Francisco
View info url is valid!
Found 7 tables in the html file
Converting html table to pandas dataframe
<table><tbody><tr><th scope="col" width="70%">Description</th>
<th scope="col" width="30%"> </th>
</tr><tr><td>
<p>Does the program or institution require students, trainees, and/or staff (faculty) to comply with specific policies or practices related to the institution’s affiliation or purpose? Such policies or practices may include, but are not limited to, admissions, hiring, retention policies, and/or requirements for completion that express mission and values?</p>
</td>
<td>
<p>_______  Yes</p>
<p>___X___  No</p>
</td>
</tr></tbody></table>


In [26]:
excel_file = "scrap/result/apa_programs_licensure.xlsx"
df = pd.read_excel(excel_file)
for ind, row in df.iterrows():
    if not row['has_pdf']:
        if row['special'] is not np.nan:
            continue
        if row['html_table'] == 1:
            file_name = f"{row['Index']:03d}-{row['University']}-{row['City']}"
            print(f"Processing row {file_name}")
            found = get_table(row['info_href'], file_name)
            if found:
                df.loc[ind, "table_found"] = 1
            # if tables:
            #     df.loc[ind, "special"] = 1
# save the updated dataframe
df.to_excel(excel_file, index=False)

Processing row 003-California Northstate University-Rancho Cordova
View info url is valid!
Processing row 004-The Chicago School, Anaheim-Anaheim
View info url is valid!
Processing row 005-University of San Francisco-San Francisco
View info url is valid!
Processing row 006-The Chicago School, Los Angeles-Los Angeles
View info url is valid!
Processing row 009-California Lutheran University-Oxnard
View info url is valid!
Processing row 025-George Washington University-Washington
View info url is valid!
Processing row 026-The Chicago School of Professional Psychology - Washington, D.C. Campus-Washington
View info url is valid!
Processing row 031-Florida Institute of Technology-Melbourne
View info url is valid!
Processing row 036-The Chicago School-Chicago
View info url is valid!
Processing row 040-The Chicago School, Chicago Campus-Chicago
View info url is valid!
Processing row 046-The Chicago School-New Orleans
View info url is valid!
Processing row 047-Springfield College-Springfield
Vi

## Parse PDF

In [ ]:
import warnings

# Ignore specific FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning, module="tabula.io")

In [ ]:
verified_excel = "scrap/result/apa_programs_licensure.xlsx"
df = pd.read_excel(verified_excel)

programs = dict()
for index, row in df.iterrows():
    if row["has_pdf"]:
        program = ProgramPDF(row)
        program.extract_page_info()
        programs[index] = program
        program.print_pdf()

In [ ]:
verified_excel = "scrap/result/apa_programs_verified.xlsx"
df = pd.read_excel(verified_excel)
from scrap.common import ProgramPDF

df_pdf = df[df["has_pdf"] == True]
program_pdf = ProgramPDF(df_pdf.iloc[0])
program_pdf.extract_page_info()
program_pdf.print_pdf()

In [ ]:
verified_excel = "scrap/result/apa_programs_verified.xlsx"
df = pd.read_excel(verified_excel)
df_pdf = df[df["has_pdf"] == True]
from scrap.common import ProgramPDF

program_pdf = ProgramPDF(df_pdf.iloc[0])

program_pdf.extract_page_info()
program_pdf.print_pdf()
program_pdf.extract_program_info()

In [ ]:
program_pdf.licensure.iloc[-1,-1]

## QA model

In [ ]:
import os
import pandas as pd
from transformers import pipeline 
import warnings
# Ignore specific FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")

# qa = pipeline("document-question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad", revision="main")
# qa = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad", revision="626af31")
# qa = pipeline("question-answering")

table_keywords = {
    "time_to_complete": ("What is the Median number of years to complete the program?",),
    "tuition": ("What is the tuition for full-time out-of-state students?",),
    "internship": ("How many students obtained APPIC member internships that were not APA/CPA-accredited?",),
    "attrition": ("How many new students in the recent cohort",),
    "licensure": ("The total number of program graduates between 2 and 10 years ago?", "How many of them became licensed psychologists?",  "What is the Licensure percentage?",),
}

excel_folder = r"scrap/materials/002-National University, Pleasant Hill, CA-John F. Kennedy University Teach-Out-Pleasant Hill"
excel_folder = r"scrap/materials/013-Fuller Theological Seminary-Pasadena"
fnames = os.listdir(excel_folder)
summary = dict()
for fname in fnames:
    if fname.endswith(".csv"):
        excel_file = os.path.join(excel_folder, fname)
        print(fname.split(".")[0])
        questions = table_keywords[fname.split(".")[0]]  
        # Read all the sheets in the excel file to dataframes and concatenate them
        df = pd.concat(pd.read_excel(excel_file, sheet_name=None), ignore_index=True)
        table_string = df.to_string(index=False)
        print(table_string)
summary

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf("your_document.pdf")

In [ ]:
print(table_string.replace("NaN", " "))  

In [ ]:
excel_file = os.path.join(excel_folder, fname)
df = pd.read_excel(excel_file)
for col in df.columns:
    df[col] = df[col].str.replace("  ", " ")
# clean up the table by removing NaN and extra spaces
table_string = df.to_string(index=False)
print(table_string)
table_string = table_string.replace("\n", " ").replace("NaN", " ")
table_string

In [ ]:
import fitz
import PyPDF2
import PyPDF2
from PyPDF2.errors import PdfReadError  # Ensure you're using a version of PyPDF2 that includes this exception

# file_name =r"scrap/pdfs/"
file_name = r"scrap/pdfs/002-National University, Pleasant Hill, CA-John F. Kennedy University Teach-Out-Pleasant Hill.pdf"
try:
    with open(file_name, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        content = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            content += page.extract_text()
except FileNotFoundError:
    print("File not found.")
except PdfReadError:
    print("Error reading PDF file.")

# save the content to a text file
text_filename = file_name.replace(".pdf", ".txt")
with open(text_filename, "w") as f:
    f.write(content)

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text
pdf_filename = os.path.join(
    "scrap",
    "pdfs",
    f"002-National University, Pleasant Hill, CA-John F. Kennedy University Teach-Out-Pleasant Hill.pdf",
)
pdf_text = extract_text_from_pdf(pdf_filename)
# save the content to a text file
text_filename = file_name.replace(".pdf", "_pitz.txt")
with open(text_filename, "w") as f:
    f.write(pdf_text)